<a href="https://colab.research.google.com/github/CivilKen/Image-Sentiment-Classification/blob/master/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!ls "/content/gdrive/My Drive/ML_homework/Colab Notebooks/hw3"

Mounted at /content/gdrive
Code		  df_valid_feature	nor_df_valid_feature  train.csv
df_test_feature   df_valid_label	StoredModel	      train_feature.csv
df_train_feature  nor_df_test_feature	test.csv
df_train_label	  nor_df_train_feature	test_feature.csv


In [0]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from skimage import io 
import matplotlib.pyplot as plt
path = "/content/gdrive/My Drive/ML_homework/Colab Notebooks/hw3/"

In [3]:
#test GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
#read dataframe 
df_train_feature = pd.read_pickle(path+"df_train_feature")
df_valid_feature = pd.read_pickle(path+"df_valid_feature")
df_test_feature = pd.read_pickle(path+"df_test_feature")
df_train_label = pd.read_pickle(path+"df_train_label")
df_valid_label = pd.read_pickle(path+"df_valid_label")
#nor_df_train_feature = pd.read_pickle(path+"nor_df_train_feature")
#nor_df_valid_feature = pd.read_pickle(path+"nor_df_valid_feature")
#nor_df_test_feature = pd.read_pickle(path+"nor_df_test_feature")

In [0]:
# parse label data
def one_hot(df_label,lenth):
  #original input is dataframe
  array_label = np.zeros([lenth,7])
  for i in range(lenth):
    idx = df_label.loc[i]
    array_label[i, idx]=1
  return array_label

arr_train_label = one_hot(df_train_label, df_train_label.shape[0])
arr_valid_label = one_hot(df_valid_label, df_valid_label.shape[0])

In [6]:
'''
# define accuracy
def accuracy(labels, pred):
  #both labels and pred are in the shape of one-hot encoding
  #both are in the format of array
  accu=0
  for i in range(len(labels)):
    lab_val = np.argmax(labels[i,:])
    pred_val = np.argmax(pred[i,:])
    if lab_val==pred_val:
      accu+=1
  return accu/len(labels)
'''

'\n# define accuracy\ndef accuracy(labels, pred):\n  #both labels and pred are in the shape of one-hot encoding\n  #both are in the format of array\n  accu=0\n  for i in range(len(labels)):\n    lab_val = np.argmax(labels[i,:])\n    pred_val = np.argmax(pred[i,:])\n    if lab_val==pred_val:\n      accu+=1\n  return accu/len(labels)\n'

In [0]:
#define hidden layer
def dense(inputs, in_size, out_size, drop_rate, activation_function=None, Wname=None, banme=None):
    # add one more layer and return the output of this layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size],stddev=0.01))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    #Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    if activation_function is None:
        outputs = Wx_plus_b
    elif activation_function=='maxout':
        outputs = tf.contrib.layers.maxout(Wx_plus_b,out_size)
    else:
        outputs = activation_function(Wx_plus_b)
    outputs = tf.nn.dropout(x=outputs, rate=drop_rate)
    return outputs, Weights, biases

In [0]:
def bottleneck_block(inputs, filters, in_size, train_mode):
  """
  Implementation of the identity block.(Identity Shortcut)
  It's used when the shortcut has the same dimension.
  當sortcut的維度相等時使用
  Arguments:
  inputs: the input tensor, [batch, width, height, input_channel, filter_numbers]
  filters: a int list with size of 3 elements, the filter number in each convolutional layer,
           the last value would be output filter size
  in_size: the input filter numbers
  train_mode: a boolean, whether this step is in training mode
  Returns:
  X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
  """
  if in_size==filters[2]:
    X_shortcut = inputs
  elif in_size<filters[2]:
    in_shape = tf.shape(inputs)
    concat_zeros = tf.zeros([in_shape[0], in_shape[1], in_shape[2],  filters[2]-in_size])
    X_shortcut = tf.concat([inputs, concat_zeros], axis=3)
  #first
  X = tf.layers.conv2d(inputs, filters[0], kernel_size=[1, 1], padding='same')
  X = tf.layers.batch_normalization(X, axis=3, training=train_mode)
  X = tf.nn.relu(X)
  #second
  X = tf.layers.conv2d(X, filters[1], kernel_size=[3, 3], padding='same')
  #p.s. 為了不改變畫素大小，所以padding使用same
  X = tf.layers.batch_normalization(X, axis=3, training=train_mode)
  X = tf.nn.relu(X)
  #third
  X = tf.layers.conv2d(inputs, filters[2], kernel_size=[1, 1], padding='same')
  #p.s. 當kernel sieze為1X1時，padding使用valid同樣不改變畫素大小
  X = tf.layers.batch_normalization(X, axis=3, training=train_mode)
  #final step
  add = tf.add(X, X_shortcut)
  add = tf.nn.relu(add)
  return add

In [0]:
def building_block(inputs, filters, in_size, train_mode):
  """
  Implementation of the identity block.(Identity Shortcut)
  It's used when the shortcut has the same dimension.
  當sortcut的維度相等時使用
  Arguments:
  inputs: the input tensor, [batch, width, height, input_channel, filter_numbers]
  filters: a int list with size of 3 elements, the filter number in each convolutional layer,
           the last value would be output filter size
  in_size: the input filter numbers
  train_mode: a boolean, whether this step is in training mode
  Returns:
  X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
  """
  if in_size==filters[2]:
    X_shortcut = inputs
  elif in_size<filters[2]:
    in_shape = tf.shape(inputs)
    concat_zeros = tf.zeros([in_shape[0], in_shape[1], in_shape[2],  filters[2]-in_size])
    X_shortcut = tf.concat([inputs, concat_zeros], axis=3)
  #first
  X = tf.layers.conv2d(inputs, filters[0], kernel_size=[3, 3], padding='same')
  X = tf.layers.batch_normalization(X, axis=3, training=train_mode)
  X = tf.nn.relu(X)
  #third
  X = tf.layers.conv2d(inputs, filters[1], kernel_size=[3, 3], padding='same')
  X = tf.layers.batch_normalization(X, axis=3, training=train_mode)
  #final step
  add = tf.add(X, X_shortcut)
  add = tf.nn.relu(add)
  return add

In [0]:
InputNeurons = df_train_feature.shape[1]
OutputNeurons = 7
drop_rate = 0.5  # dropout parameter, drop how many percentage
learning_rate = 0.0001
beta = 0.001 #it's used in regularization as lamda
printstep = 100
traing_epochs = 50001

# define placeholder for inputs to network
rs = tf.placeholder(tf.float32) #drop rate
ts = tf.placeholder(tf.bool) #training mode
xs = tf.placeholder(tf.float32, [None, InputNeurons])
ys = tf.placeholder(tf.float32, [None, OutputNeurons])

In [0]:
input_layer = tf.reshape(xs, [-1,48,48,1])
conv1 = tf.layers.conv2d(inputs=input_layer, filters=64, kernel_size=[3,3], padding='same', activation=tf.nn.relu)
bn1 = tf.layers.batch_normalization(conv1, axis=3, training=ts)
rl1 = tf.nn.relu(bn1)
pool1 = tf.layers.max_pooling2d(inputs=rl1, pool_size=[2,2], strides=2)
res1 = bottleneck_block(pool1, [32,32,64], 64, ts)
res2 = bottleneck_block(res1, [32,32,64], 64, ts)
res3 = bottleneck_block(res2, [64,64,128], 64, ts)
res4 = bottleneck_block(res3, [64,64,128], 128, ts)
pool2 = tf.layers.average_pooling2d(inputs=res3, pool_size=[2,2], strides=2)
pool_flat = tf.reshape(pool2, [-1, 12*12*128])
[dense1, w1, b1] = dense(pool_flat, 12*12*128, 1024, rs, activation_function=tf.nn.relu)
logits = tf.layers.dense(inputs=dense1, units=OutputNeurons)
prediction = tf.nn.softmax(logits)

In [0]:
#calculate loss function
regularizers = tf.nn.l2_loss(w1)
#loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), axis=1))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=ys, logits=logits))
loss = loss + beta*regularizers
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# start of the neural network
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

TrainLoss=[]
ValidLoss=[]
epoch=[]
for i in range(traing_epochs):
    # training
    idx = np.random.choice(np.arange(len(df_train_feature)), 64, replace=False)
    x_train_batch = df_train_feature.loc[idx,:]
    y_train_batch = arr_train_label[idx,:]
    sess.run(train_step, feed_dict={xs: x_train_batch, ys: y_train_batch, rs: drop_rate, ts: True})
    if i % printstep == 0:
        # to see the step improvement
        train_loss = sess.run(loss, feed_dict={xs: x_train_batch, ys: y_train_batch, rs: 0, ts: False})
        validation_loss = sess.run(loss, feed_dict={xs: df_valid_feature, ys: arr_valid_label, rs:0, ts: False})
        print("epoch",i,"validation loss=", validation_loss,"Train Loss=",train_loss)
        TrainLoss.append(train_loss)
        ValidLoss.append(validation_loss)
        epoch.append(i)

epoch 0 validation loss= 2.8013132 Train Loss= 2.791047
epoch 100 validation loss= 2.8203585 Train Loss= 2.8405666
epoch 200 validation loss= 2.7788196 Train Loss= 2.7769907
epoch 300 validation loss= 2.7413116 Train Loss= 2.734816
epoch 400 validation loss= 2.7147086 Train Loss= 2.7149498
epoch 500 validation loss= 2.676189 Train Loss= 2.684793
epoch 600 validation loss= 2.634014 Train Loss= 2.6372194
epoch 700 validation loss= 2.5991538 Train Loss= 2.6007788
epoch 800 validation loss= 2.5763466 Train Loss= 2.5796664
epoch 900 validation loss= 2.5488687 Train Loss= 2.5510323
epoch 1000 validation loss= 2.5266716 Train Loss= 2.5366418
epoch 1100 validation loss= 2.4978366 Train Loss= 2.4929013
epoch 1200 validation loss= 2.4745886 Train Loss= 2.4685168
epoch 1300 validation loss= 2.4535666 Train Loss= 2.4513965
epoch 1400 validation loss= 2.4324355 Train Loss= 2.4416425
epoch 1500 validation loss= 2.4027586 Train Loss= 2.3955843
epoch 1600 validation loss= 2.3968272 Train Loss= 2.39437

In [0]:
#%% plot loss
fig = plt.figure()
# 111代表在subplot圖中的位置
ax = fig.add_subplot(111)
ax.plot(epoch[1:], TrainLoss[1:], c='b', label='TrainLoss', linewidth=0.5)
ax.plot(epoch[1:], ValidLoss[1:], c='r', label='ValidLoss', linewidth=0.5)
leg = plt.legend()
# get the lines and texts inside legend box
leg_lines = leg.get_lines()
leg_texts = leg.get_texts()
# bulk-set the properties of all lines and texts
plt.setp(leg_lines, linewidth=6)
plt.setp(leg_texts, fontsize='x-large')
plt.show()

In [0]:
# Add ops to save the variables.
saver = tf.train.Saver()
save_path = saver.save(sess, path+"StoredModel/v4_model.ckpt")

In [0]:
'''
#Restore Model
sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, path+"StoredModel/v4_model.ckpt")
'''

In [0]:
arr_train_pred = sess.run(prediction, feed_dict={xs: df_train_feature, ys: arr_train_label, rs: 0, ts: False})
arr_valid_pred = sess.run(prediction, feed_dict={xs: df_valid_feature, ys: arr_valid_label, rs: 0, ts: False})

In [0]:
train_accuracy = accuracy(arr_train_label, arr_train_pred)
print("train_accuracy = ", train_accuracy*100,"%")
valid_accuracy = accuracy(arr_valid_label, arr_valid_pred)
print("valid_accuracy = ", valid_accuracy*100,"%")